In [1]:
# Step 1: Install required packages (run in notebook or script shell)
!pip install kaggle split-folders torch torchvision

# Step 2: Download and unzip the PlantVillage dataset
!kaggle datasets download -d emmarex/plantdisease -p ./data --unzip
# The dataset unzips to ./data/PlantVillage

import os

# [CHANGE 1]: Always check the extracted folder name
root_folder = "./data/PlantVillage"
if not os.path.exists(root_folder):
    raise ValueError("Dataset not found at expected location.")


Defaulting to user installation because normal site-packages is not writeable


ERROR: Could not find a version that satisfies the requirement kaggle (from versions: none)
ERROR: No matching distribution found for kaggle


HTTPSConnectionPool(host='www.kaggle.com', port=443): Max retries exceeded with url: /api/v1/datasets/metadata/emmarex/plantdisease (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x000002A63C6B2660>: Failed to resolve 'www.kaggle.com' ([Errno 11001] getaddrinfo failed)"))


In [2]:
import splitfolders


splitfolders.ratio(
    root_folder,
    output="./data/output",
    seed=42,
    ratio=(0.8, 0.2)  # 80% train, 20% val
)



Copying files: 20639 files [00:34, 605.57 files/s]


In [3]:
import os
print("Train exists?", os.path.exists("./data/output/train"))
print("Val exists?", os.path.exists("./data/output/val"))
print("Classes in train:", os.listdir("./data/output/train"))
print("Classes in val:", os.listdir("./data/output/val"))


Train exists? True
Val exists? True
Classes in train: ['Pepper__bell___Bacterial_spot', 'Pepper__bell___healthy', 'PlantVillage', 'Potato___Early_blight', 'Potato___healthy', 'Potato___Late_blight', 'Tomato_Bacterial_spot', 'Tomato_Early_blight', 'Tomato_healthy', 'Tomato_Late_blight', 'Tomato_Leaf_Mold', 'Tomato_Septoria_leaf_spot', 'Tomato_Spider_mites_Two_spotted_spider_mite', 'Tomato__Target_Spot', 'Tomato__Tomato_mosaic_virus', 'Tomato__Tomato_YellowLeaf__Curl_Virus']
Classes in val: ['Pepper__bell___Bacterial_spot', 'Pepper__bell___healthy', 'PlantVillage', 'Potato___Early_blight', 'Potato___healthy', 'Potato___Late_blight', 'Tomato_Bacterial_spot', 'Tomato_Early_blight', 'Tomato_healthy', 'Tomato_Late_blight', 'Tomato_Leaf_Mold', 'Tomato_Septoria_leaf_spot', 'Tomato_Spider_mites_Two_spotted_spider_mite', 'Tomato__Target_Spot', 'Tomato__Tomato_mosaic_virus', 'Tomato__Tomato_YellowLeaf__Curl_Virus']


In [5]:
import shutil
folders = ['./data/output/train/PlantVillage', './data/output/val/PlantVillage']
for folder in folders:
    if os.path.exists(folder):
        shutil.rmtree(folder)
        print(f"Removed: {folder}")


In [7]:
print("Train classes:", os.listdir('./data/output/train'))
print("Val classes:", os.listdir('./data/output/val'))


Train classes: ['Pepper__bell___Bacterial_spot', 'Pepper__bell___healthy', 'Potato___Early_blight', 'Potato___healthy', 'Potato___Late_blight', 'Tomato_Bacterial_spot', 'Tomato_Early_blight', 'Tomato_healthy', 'Tomato_Late_blight', 'Tomato_Leaf_Mold', 'Tomato_Septoria_leaf_spot', 'Tomato_Spider_mites_Two_spotted_spider_mite', 'Tomato__Target_Spot', 'Tomato__Tomato_mosaic_virus', 'Tomato__Tomato_YellowLeaf__Curl_Virus']
Val classes: ['Pepper__bell___Bacterial_spot', 'Pepper__bell___healthy', 'Potato___Early_blight', 'Potato___healthy', 'Potato___Late_blight', 'Tomato_Bacterial_spot', 'Tomato_Early_blight', 'Tomato_healthy', 'Tomato_Late_blight', 'Tomato_Leaf_Mold', 'Tomato_Septoria_leaf_spot', 'Tomato_Spider_mites_Two_spotted_spider_mite', 'Tomato__Target_Spot', 'Tomato__Tomato_mosaic_virus', 'Tomato__Tomato_YellowLeaf__Curl_Virus']


In [8]:
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

# [CHANGE 3]: Use splitfolders output for ImageFolder
data_dir = "./data/output"

transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
    transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
])

train_dataset = datasets.ImageFolder(os.path.join(data_dir, "train"), transform=transform)
val_dataset   = datasets.ImageFolder(os.path.join(data_dir, "val"),   transform=transform)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=2)
val_loader   = DataLoader(val_dataset, batch_size=32, shuffle=False, num_workers=2)

num_classes = len(train_dataset.classes)
print("Classes:", train_dataset.classes)
print("Number of classes:", num_classes)


Classes: ['Pepper__bell___Bacterial_spot', 'Pepper__bell___healthy', 'Potato___Early_blight', 'Potato___Late_blight', 'Potato___healthy', 'Tomato_Bacterial_spot', 'Tomato_Early_blight', 'Tomato_Late_blight', 'Tomato_Leaf_Mold', 'Tomato_Septoria_leaf_spot', 'Tomato_Spider_mites_Two_spotted_spider_mite', 'Tomato__Target_Spot', 'Tomato__Tomato_YellowLeaf__Curl_Virus', 'Tomato__Tomato_mosaic_virus', 'Tomato_healthy']
Number of classes: 15


In [9]:
print(len(train_dataset), len(val_dataset))


16504 4134


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

class PlantDiseaseCNN(nn.Module):
    def __init__(self, num_classes):
        super(PlantDiseaseCNN, self).__init__()
        self.conv_layers = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),
            nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),
            nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2)
        )
        self.fc_layers = nn.Sequential(
            nn.Flatten(),
            nn.Linear(128 * 16 * 16, 256),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(256, num_classes)
        )
    def forward(self, x):
        x = self.conv_layers(x)
        x = self.fc_layers(x)
        return x

model = PlantDiseaseCNN(num_classes).to(device)
print(model)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

epochs = 10
best_acc = 0.0
for epoch in range(epochs):
    model.train()
    running_loss, correct, total = 0.0, 0, 0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        correct += (predicted == labels).sum().item()
        total += labels.size(0)
    train_acc = 100 * correct / total

    # Validation
    model.eval()
    val_correct, val_total = 0, 0
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            val_correct += (predicted == labels).sum().item()
            val_total += labels.size(0)
    val_acc = 100 * val_correct / val_total
    print(f"Epoch [{epoch+1}/{epochs}] Loss: {running_loss/len(train_loader):.4f} | Train Acc: {train_acc:.2f}% | Val Acc: {val_acc:.2f}%")
    if val_acc > best_acc:
        best_acc = val_acc
        torch.save(model.state_dict(), "best_plant_disease_model.pth")
        print("✅ Saved Best Model")
print("Training Finished. Best Validation Accuracy:", best_acc)


Using device: cpu
PlantDiseaseCNN(
  (conv_layers): Sequential(
    (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): ReLU()
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU()
    (8): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (fc_layers): Sequential(
    (0): Flatten(start_dim=1, end_dim=-1)
    (1): Linear(in_features=32768, out_features=256, bias=True)
    (2): ReLU()
    (3): Dropout(p=0.5, inplace=False)
    (4): Linear(in_features=256, out_features=15, bias=True)
  )
)


In [53]:
from PIL import Image
import torch
from torchvision import transforms

# 1. Load saved model
model = PlantDiseaseCNN(num_classes)  # Define the model architecture
model.load_state_dict(torch.load("best_plant_disease_model.pth"))
model.eval()  # Set model to evaluation mode

# 2. Define transform (same as during training)
transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
    transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
])

# 3. Load and preprocess a new image
img_path = 'C:/Users/Nitin Dumka/OneDrive/Desktop/Tomato_bacterial_spot01.jpg'
image = Image.open(img_path).convert('RGB')
input_tensor = transform(image).unsqueeze(0)  # Add batch dimension

# 4. Predict disease
with torch.no_grad():
    outputs = model(input_tensor)
    _, predicted_class = torch.max(outputs, 1)

# 5. Map prediction to label
class_names = train_dataset.classes  # List of class labels from your dataset
predicted_label = class_names[predicted_class.item()]

print(f"Predicted disease: {predicted_label}")


Predicted disease: Tomato_Septoria_leaf_spot


In [1]:
import sys
print(sys.executable)



c:\Users\Nitin Dumka\ml_gpu_env\Scripts\python.exe


In [3]:
import torch
import tensorflow as tf
import sklearn

print("Torch CUDA available:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("GPU Name:", torch.cuda.get_device_name(0))

print("TensorFlow GPUs:", tf.config.list_physical_devices('GPU'))
print("scikit-learn version:", sklearn.__version__)



Torch CUDA available: False
TensorFlow GPUs: []
scikit-learn version: 1.7.1
